In [1]:
import cv2
import os
import numpy as np

In [4]:
source_Image_Path = r'C:\Users\Sachin13390\Desktop\Archive'

destination_Image_Path = r'C:\Users\Sachin13390\Desktop\Resized Image'

In [6]:
def getNewResizedImage(input_Image, Image_Size):

    height,width = input_Image.shape
    #print (height, width)

    if width > height:
        aspect_Ratio = (float)(width/height)
        width = 20
        height = int(round(width/aspect_Ratio))
    else:
        aspect_Ratio = (float)(height/width)
        height = 20
        width = int(round(height/aspect_Ratio))

    input_Image = cv2.resize(input_Image, (width,height), interpolation = cv2.INTER_AREA )
    
    height,width = input_Image.shape
    

    number_Of_Column_To_Add = 28-width
    temp_Column = np.zeros( (height , int(number_Of_Column_To_Add/2)), dtype = np.uint8)
    input_Image = np.append(temp_Column, input_Image, axis=1)
    input_Image = np.append(input_Image, temp_Column, axis=1)

    height,width = input_Image.shape


    number_Of_Row_To_Add = 28-height
    temp_Row= np.zeros( (int(number_Of_Row_To_Add/2) , width ), dtype = np.uint8)
    input_Image = np.concatenate((temp_Row,input_Image))
    input_Image = np.concatenate((input_Image,temp_Row))

    return cv2.resize(input_Image, (Image_Size,Image_Size), interpolation = cv2.INTER_AREA )

In [76]:
os.chdir(source_Image_Path)

for root, dirs, files in os.walk(".", topdown = False):
    if not os.path.isdir(os.path.join(destination_Image_Path,root.replace(".\\",""))):
        os.mkdir(os.path.join(destination_Image_Path,root.replace(".\\","")))
        
    number = 1

    for filename in files:
        if filename.endswith(('.jpg')):
            Image_File_Path  = os.path.join(source_Image_Path,root.replace(".\\",""),filename)
            #print (Image_File_Path)

            image = cv2.imread(Image_File_Path,0)

            im2,contours,hierarchy = cv2.findContours(image,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

            Biggest_Countour = max(contours, key = cv2.contourArea)
            x,y,w,h = cv2.boundingRect(Biggest_Countour)
            image = cv2.imread(Image_File_Path,0)

            #image = cv2.bitwise_not(image)
            resize_image =image[y:y+h, x:x+w]
            resize_image = getNewResizedImage(resize_image,28)

            file_Path_to_Save_resized_Image = os.path.join(destination_Image_Path,root.replace(".\\",""),filename)
            
            #print (file_Path_to_Save_resized_Image)
            cv2.imwrite(file_Path_to_Save_resized_Image, resize_image)
            
            number = number + 1          
            if number%100==0:
                print ("Processing "+str(root.replace(".\\",""))+" : "+str(number))
                
                